In [2]:
import os
import json 
import random
import numpy as np
from PIL import Image
import cv2 

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from tqdm import tqdm
import copy

# ⭐️ Albumentations 및 PyTorch 바인딩 임포트
import albumentations as A
from albumentations.pytorch import ToTensorV2 

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report
)

# Deterministic 연산을 위한 환경 변수 강제 설정
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

# 시드 고정 함수
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False 

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Device set to: {device}")

env: CUBLAS_WORKSPACE_CONFIG=:4096:8
✅ Device set to: cuda


In [3]:
# [Cell 2] 경로 및 하이퍼파라미터 설정

# ★ 경로 설정 (사용자 환경 경로) ★
DATA_ROOT = "/workspace/user4/cropped" 
TRAIN_IMG_ROOT = os.path.join(DATA_ROOT, "train")
VAL_IMG_ROOT = os.path.join(DATA_ROOT, "val")

# 하이퍼파라미터
BATCH_SIZE = 32
NUM_WORKERS = 4 
NUM_EPOCHS = 50
LEARNING_RATE = 1e-4

In [4]:
# [Cell 2]에서 경로 설정 후, 이 셀을 실행하여 실제 파일 유무를 확인하세요.
print(f"Train Image Root: {TRAIN_IMG_ROOT}")
print(f"Val Image Root: {VAL_IMG_ROOT}")

# 폴더 존재 여부 확인
print(f"Train 폴더 존재 여부: {os.path.isdir(TRAIN_IMG_ROOT)}")
print(f"Val 폴더 존재 여부: {os.path.isdir(VAL_IMG_ROOT)}")

# 하위 감정 폴더 존재 여부 확인
print(f"Train/happy 폴더 존재 여부: {os.path.isdir(os.path.join(TRAIN_IMG_ROOT, 'happy'))}")
print(f"Train/anger 폴더 존재 여부: {os.path.isdir(os.path.join(TRAIN_IMG_ROOT, 'anger'))}")
print(f"Train/sadness 폴더 존재 여부: {os.path.isdir(os.path.join(TRAIN_IMG_ROOT, 'sadness'))}")
print(f"Val/happy 폴더 존재 여부: {os.path.isdir(os.path.join(VAL_IMG_ROOT, 'happy'))}")

Train Image Root: /workspace/user4/cropped/train
Val Image Root: /workspace/user4/cropped/val
Train 폴더 존재 여부: True
Val 폴더 존재 여부: True
Train/happy 폴더 존재 여부: True
Train/anger 폴더 존재 여부: True
Train/sadness 폴더 존재 여부: True
Val/happy 폴더 존재 여부: True


In [5]:
## 데이터 유효성 검증🔎
def validate_image_data(root_dir):
    print(f"🔍 데이터 검증 시작: {root_dir}")
    
    total_files = 0
    corrupt_files = 0
    
    valid_ext = ('.jpg', '.jpeg', '.png', '.bmp')
    
    for root, dirs, files in os.walk(root_dir):
        image_files = [f for f in files if f.lower().endswith(valid_ext)]
        
        if len(image_files) > 0:
            print(f"  📁 Folder: {os.path.basename(root)} | Images: {len(image_files)}")
            
            for img_file in tqdm(image_files, desc=f"Checking {os.path.basename(root)}", leave=False):
                img_path = os.path.join(root, img_file)
                try:
                    with Image.open(img_path) as img:
                        img.verify()
                except (IOError, SyntaxError) as e:
                    print(f"    ❌ Corrupt file found: {img_path}")
                    corrupt_files += 1
                
                total_files += 1

    print("-" * 40)
    print(f"✅ 검증 완료")
    print(f"   - 총 이미지 수: {total_files}")
    print(f"   - 손상된 파일 수: {corrupt_files}")
    
    if corrupt_files > 0:
        print("⚠️ 경고: 손상된 파일이 존재합니다. 해당 파일을 제거하거나 확인해주세요.")
    else:
        print("🎉 모든 파일이 정상입니다.")

# 검증 실행
if os.path.exists(DATA_ROOT):
    validate_image_data(DATA_ROOT)
else:
    print(f"❌ 경로 오류: {DATA_ROOT} 가 존재하지 않습니다.")

🔍 데이터 검증 시작: /workspace/user4/cropped
  📁 Folder: val_sadness_invalid | Images: 1


  📁 Folder: sadness | Images: 280


  📁 Folder: panic | Images: 280


  📁 Folder: val_anger_invalid | Images: 1


  📁 Folder: anger | Images: 290


...

  📁 Folder: train_happy_invalid | Images: 8


  📁 Folder: anger | Images: 1394


  📁 Folder: sadness | Images: 1528


  📁 Folder: happy | Images: 5245


----------------------------------------
✅ 검증 완료
   - 총 이미지 수: 10701
   - 손상된 파일 수: 0
🎉 모든 파일이 정상입니다.


In [6]:
# [Cell 4] EmotionDataset 클래스 정의 (Albumentations + OpenCV + New Folders)
class EmotionDataset(Dataset):
    """
    OpenCV (cv2)로 이미지를 로드하고, NumPy 배열 상태 그대로 반환하며,
    'anger', 'happy', 'panic', 'sadness' 4개 폴더를 이진 분류(happy: 1)로 매핑합니다.
    """
    def __init__(self, img_root, transform=None):
        self.img_root = img_root
        self.transform = transform
        
        # ⭐️ 스캔할 폴더 목록 (Invalid 폴더 제외)
        self.target_emotions = ["anger", "happy", "panic", "sadness"] 
        
        # ⭐️ 이진 분류 레이블 매핑: happy는 1, 나머지는 0 (Others)
        self.label_map = {"happy": 1, "anger": 0, "panic": 0, "sadness": 0} 
        self.samples = []
        
        valid_ext = ('.jpg', '.jpeg', '.png', '.bmp')

        print(f"\n[Dataset Loading: {os.path.basename(img_root)}...]")
        
        # ⭐️ target_emotions에 정의된 4개 폴더만 반복
        for emo in self.target_emotions: 
            emo_dir = os.path.join(self.img_root, emo) # 예: /cropped/train/anger
            
            if not os.path.isdir(emo_dir):
                print(f"🚨 FAILED: 폴더가 없습니다: {emo_dir}") 
                continue
                
            label = self.label_map[emo]
            count = 0
            
            for filename in os.listdir(emo_dir):
                if filename.lower().endswith(valid_ext):
                    img_path = os.path.join(emo_dir, filename)
                    self.samples.append((img_path, label))
                    count += 1
            
            print(f"🔍 CHECK: {emo_dir} ({'Happy' if label==1 else 'Others'}) 에서 {count}개 찾음") 
            
        random.shuffle(self.samples)
        print(f"  >> Total samples loaded: {len(self.samples)}개")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        try:
            # 1. cv2.imread()를 사용하여 이미지 로드 (NumPy 배열, BGR)
            image = cv2.imread(img_path)
            
            if image is None:
                raise FileNotFoundError(f"Failed to load image with cv2: {img_path}")
            
            # 2. BGR을 RGB로 변환 (Albumentations 호환)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
        except Exception as e:
            print(f"Error loading/converting image {img_path}: {e}. Skipping to index 0.")
            return self.__getitem__(0)

        # 3. Albumentations transforms 적용 (NumPy 배열 입력)
        if self.transform is not None:
            augmented = self.transform(image=image) 
            image = augmented['image']

        label = torch.tensor(label, dtype=torch.long)
        return image, label

In [7]:
# [Cell 5] Albumentations Transforms 및 DataLoader 정의

# ⭐️ Albumentations 파이프라인 정의 (NumPy 배열 입력) ⭐️

# 이미지 평균/표준편차 (ResNet ImageNet 기준)
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_transform = A.Compose([
    # 데이터 증강 (Resize는 필수)
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    # A.ShiftScaleRotate(p=0.5, scale_limit=0.1, rotate_limit=10), # 추가 증강 예시
    
    # 정규화 (Normalization)
    A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD, max_pixel_value=255.0),
    
    # 텐서 변환 (HWC -> CHW 변환 및 float32로 자동 변환)
    ToTensorV2(), 
])

val_transform = A.Compose([
    A.Resize(224, 224),
    
    # 정규화
    A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD, max_pixel_value=255.0),
    
    # 텐서 변환
    ToTensorV2(),
])

# 데이터셋 생성
train_dataset = EmotionDataset(TRAIN_IMG_ROOT, transform=train_transform)
val_dataset   = EmotionDataset(VAL_IMG_ROOT, transform=val_transform)

# 데이터로더 생성 (pin_memory=True 유지)
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=NUM_WORKERS,
    pin_memory=True 
)
val_loader   = DataLoader(
    val_dataset,   
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=NUM_WORKERS,
    pin_memory=True 
)

print("\n✅ DataLoaders and Albumentations are ready.")


[Dataset Loading: train...]
🔍 CHECK: /workspace/user4/cropped/train/anger (Others) 에서 1394개 찾음
🔍 CHECK: /workspace/user4/cropped/train/happy (Happy) 에서 5245개 찾음
🔍 CHECK: /workspace/user4/cropped/train/panic (Others) 에서 1368개 찾음
🔍 CHECK: /workspace/user4/cropped/train/sadness (Others) 에서 1528개 찾음
  >> Total samples loaded: 9535개

[Dataset Loading: val...]
🔍 CHECK: /workspace/user4/cropped/val/anger (Others) 에서 290개 찾음
🔍 CHECK: /workspace/user4/cropped/val/happy (Happy) 에서 299개 찾음
🔍 CHECK: /workspace/user4/cropped/val/panic (Others) 에서 280개 찾음
🔍 CHECK: /workspace/user4/cropped/val/sadness (Others) 에서 280개 찾음
  >> Total samples loaded: 1149개

✅ DataLoaders and Albumentations are ready.


In [8]:
## CBAM 모듈 및 ResNetCBAM 모델 정의 (⭐️ CBAM 추가됨)CBAM 모듈 및 ResNetCBAM 모델 정의 (⭐️ CBAM 추가됨)
## CBAM 모듈과, ResNet에 CBAM을 통합한 ResNetCBAM 모델을 정의합니다.

print("🛠️ CBAM 모듈 및 ResNetCBAM 모델 설정 중...")

# ⭐️ 채널 어텐션 모듈 (Channel Attention Module) 정의
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
                                nn.ReLU(),
                                nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

# ⭐️ 공간 어텐션 모듈 (Spatial Attention Module) 정의
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_out = torch.cat([avg_out, max_out], dim=1)
        x_out = self.conv1(x_out)
        return self.sigmoid(x_out)

# ⭐️ CBAM (Channel and Spatial Attention Module) 정의
class CBAM(nn.Module):
    def __init__(self, in_planes, ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.ca = ChannelAttention(in_planes, ratio)
        self.sa = SpatialAttention(kernel_size)

    def forward(self, x):
        out = x * self.ca(x)
        out = out * self.sa(out)
        return out

# ⭐️ ResNet18 백본에 CBAM을 통합한 모델 정의
class ResNetCBAM(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNetCBAM, self).__init__()
        
        # 1. 사전학습된 ResNet18 로드
        resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        
        # ResNet의 초기 Conv1, bn1, relu, maxpool을 그대로 사용
        self.conv1 = resnet.conv1
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        
        # ResNet의 각 Stage (layer1, layer2, layer3, layer4)를 가져옴
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4
        
        # ⭐️ 각 Residual Block 이후 또는 Layer 이후에 CBAM 모듈 삽입 ⭐️
        # ResNet18의 각 레이어의 출력 채널에 맞춰 CBAM을 초기화합니다.
        #self.cbam1 = CBAM(64)  # layer1의 출력 채널 (ResNet18)
        self.cbam2 = CBAM(128) # layer2의 출력 채널
        self.cbam3 = CBAM(256) # layer3의 출력 채널
        #self.cbam4 = CBAM(512) # layer4의 출력 채널
        ## CBAM 모듈은 stage 2,3에만 적용
        
        self.avgpool = resnet.avgpool
        
        # 최종 분류기를 2클래스로 교체
        num_features = resnet.fc.in_features
        self.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.cbam1(x) # ⭐️ CBAM 삽입 지점
        x = self.layer2(x)
        x = self.cbam2(x) # ⭐️ CBAM 삽입 지점
        x = self.layer3(x)
        x = self.cbam3(x) # ⭐️ CBAM 삽입 지점
        x = self.layer4(x)
        x = self.cbam4(x) # ⭐️ CBAM 삽입 지점
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)# ⭐️ ResNet18 백본에 CBAM을 Stage 2와 3에만 통합한 모델 정의
class ResNetCBAM(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNetCBAM, self).__init__()
        
        # 1. 사전학습된 ResNet18 로드
        resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        
        # ResNet의 초기 Conv1, bn1, relu, maxpool을 그대로 사용
        self.conv1 = resnet.conv1
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        
        # ResNet의 각 Stage (layer1, layer2, layer3, layer4)를 가져옴
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4
        
        # ⭐️ Stage 2와 Stage 3의 출력 채널에 맞춰 CBAM 초기화
        self.cbam2 = CBAM(128) # layer2의 출력 채널
        self.cbam3 = CBAM(256) # layer3의 출력 채널
        
        # self.cbam1과 self.cbam4는 정의하지 않음
        
        self.avgpool = resnet.avgpool
        
        # 최종 분류기를 2클래스로 교체
        num_features = resnet.fc.in_features
        self.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        # x = self.cbam1(x) # ⭐️ Stage 1 CBAM 제거
        
        x = self.layer2(x)
        x = self.cbam2(x) # ⭐️ Stage 2 CBAM 유지
        
        x = self.layer3(x)
        x = self.cbam3(x) # ⭐️ Stage 3 CBAM 유지
        
        x = self.layer4(x)
        # x = self.cbam4(x) # ⭐️ Stage 4 CBAM 제거
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# 모델, 손실함수, 최적화함수 초기화
model = ResNetCBAM(num_classes=2).to(device) # ResNetCBAM 모델 사용
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)

print("✅ ResNetCBAM Model (CBAM @ Stage 2, 3) is ready!")

🛠️ CBAM 모듈 및 ResNetCBAM 모델 설정 중...


✅ ResNetCBAM Model (CBAM @ Stage 2, 3) is ready!


In [9]:
## 학습 함수 정의 ## 
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 30)

        # ---- Train ----
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total = 0

        for inputs, labels in tqdm(train_loader, desc="Training", leave=False):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)
        
        # ⭐️ 학습 후 GPU 캐시 메모리 정리
        if device.type == 'cuda':
            torch.cuda.empty_cache()

        epoch_loss = running_loss / total
        epoch_acc = running_corrects.double() / total
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        val_total = 0

        with torch.inference_mode():
            for inputs, labels in tqdm(val_loader, desc="Validation", leave=False):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)
                val_total += labels.size(0)
        
        # ⭐️ 검증 후 GPU 캐시 메모리 정리
        if device.type == 'cuda':
            torch.cuda.empty_cache()

        val_loss /= val_total
        val_acc = val_corrects.double() / val_total
        print(f"Val   Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        # Best Model 저장
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, "Res_CBAM_v3_epc50.pth") # ⭐️ 파일명 변경
            print(f"⭐ Best model saved! (Acc: {best_val_acc:.4f})")

    print(f"\nTraining Complete. Best Val Acc: {best_val_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model

In [10]:
## 학습 실행
# 학습 시작
trained_model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=NUM_EPOCHS)


Epoch 1/50
------------------------------


Training:   0%|          | 0/298 [00:00<?, ?it/s]

Train Loss: 0.1429 Acc: 0.9451


Val   Loss: 0.1092 Acc: 0.9678
⭐ Best model saved! (Acc: 0.9678)

Epoch 2/50
------------------------------


Train Loss: 0.0526 Acc: 0.9822


...

Val   Loss: 0.1537 Acc: 0.9643

Epoch 49/50
------------------------------


Train Loss: 0.0047 Acc: 0.9980


Val   Loss: 0.1952 Acc: 0.9495

Epoch 50/50
------------------------------


Train Loss: 0.0036 Acc: 0.9988


Val   Loss: 0.1514 Acc: 0.9678

Training Complete. Best Val Acc: 0.9826


In [ ]:
## 성능 평가
print("\n📊 최종 성능 평가 중...")

trained_model.eval()
all_labels = []
all_preds = []

with torch.inference_mode():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = trained_model(inputs)
        _, preds = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

# Metrics 계산
acc  = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, pos_label=1)
rec  = recall_score(all_labels, all_preds, pos_label=1)
f1   = f1_score(all_labels, all_preds, pos_label=1)
cm   = confusion_matrix(all_labels, all_preds)

print("\n=== Val Metrics (binary: 1=HAPPY, 0=OTHERS) ===")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-score : {f1:.4f}")
print("\nConfusion Matrix (rows: true, cols: pred)")
print(cm)

print("\nClassification report:")
print(classification_report(all_labels, all_preds, target_names=["OTHERS", "HAPPY"]))


📊 최종 성능 평가 중...


Evaluating: 100%|██████████| 36/36 [00:04<00:00,  8.56it/s]


=== Val Metrics (binary: 1=HAPPY, 0=OTHERS) ===
Accuracy : 0.9800
Precision: 0.9600
Recall   : 0.9632
F1-score : 0.9616

Confusion Matrix (rows: true, cols: pred)
[[838  12]
 [ 11 288]]

Classification report:
              precision    recall  f1-score   support

      OTHERS       0.99      0.99      0.99       850
       HAPPY       0.96      0.96      0.96       299

    accuracy                           0.98      1149
   macro avg       0.97      0.97      0.97      1149
weighted avg       0.98      0.98      0.98      1149



: 